In [1]:
#%%
import pandas as pd
import numpy as np
import os
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pickle
import json
from tqdm import tqdm
from glob import glob
from multiprocessing import Pool
import time

region = "ridgecrest"
#region = "project_chile"
#region = "testing"
data_path = lambda x: os.path.join(f"{region}", x)

stations = pd.read_csv(data_path("stations.csv"))
picks = pd.read_csv(data_path("gamma_picks.csv"))
events = pd.read_csv(data_path("gamma_events.csv"))


# Convert Stations

In [2]:
converted_hypoinverse = []
converted_hypoDD = {}

for i in tqdm(range(len(stations))):

    network_code, station_code, comp_code, channel_code = stations.iloc[i]['id'].split('.')
    station_weight = " "
    lat_degree = int(stations.iloc[i]['latitude'])
    lat_minute = (stations.iloc[i]['latitude'] - lat_degree) * 60
    north = "N" if lat_degree >= 0 else "S"
    lng_degree = int(stations.iloc[i]['longitude'])
    lng_minute = (stations.iloc[i]['longitude'] - lng_degree) * 60
    west = "W" if lng_degree <= 0 else "E"
    elevation = stations.iloc[i]['elevation_m']
    line_hypoinverse = f"{station_code:<5} {network_code:<2} {comp_code[:-1]:<1}{channel_code:<3} {station_weight}{abs(lat_degree):2.0f} {abs(lat_minute):7.4f}{north}{abs(lng_degree):3.0f} {abs(lng_minute):7.4f}{west}{elevation:4.0f}\n"
    # line_hypoDD = f"{network_code:<2}.{station_code:<5} {stations.iloc[i]['latitude']:.3f}, {stations.iloc[i]['longitude']:.3f}\n"
    #line_hypoDD = f"{station_code} {stations.iloc[i]['latitude']:.3f} {stations.iloc[i]['longitude']:.3f}\n"
    converted_hypoinverse.append(line_hypoinverse)
    #converted_hypoDD.append(line_hypoDD)
    converted_hypoDD[f"{station_code}"] = f"{station_code} {stations.iloc[i]['latitude']:.3f} {stations.iloc[i]['longitude']:.3f}\n"

# %%
out_file = 'results/stations_hypoinverse.dat'
with open(out_file, 'w') as f:
    f.writelines(converted_hypoinverse)

out_file = 'results/stations_hypoDD.dat'
# converted_hypoDD = list(set(converted_hypoDD))
with open(out_file, 'w') as f:
    #f.writelines(converted_hypoDD)
    for k, v in converted_hypoDD.items():
        f.write(v)

# %%

100%|████████████████████████████████████████████████████████████████████████████████| 58/58 [00:00<00:00, 1067.04it/s]


# GaMMa to Hypoinverse

In [3]:
events["match_id"] = events.apply(lambda x: f'{x["event_index"]}_{x["event_index"]}', axis=1)
picks["match_id"] = picks.apply(lambda x: f'{x["event_index"]}_{x["event_index"]}', axis=1)

# %%
out_file = open("results/hypoInput.arc", "w")

picks_by_event = picks.groupby("match_id").groups

for i in tqdm(range(len(events))):

    event = events.iloc[i]
    event_time = datetime.strptime(event["time"], "%Y-%m-%dT%H:%M:%S.%f").strftime("%Y%m%d%H%M%S%f")[:-4]
    lat_degree = int(event["latitude"])
    lat_minute = (event["latitude"] - lat_degree) * 60 * 100
    south = "S" if lat_degree <= 0 else " "
    lng_degree = int(event["longitude"])
    lng_minute = (event["longitude"] - lng_degree) * 60 * 100
    east = "E" if lng_degree >= 0 else " "
    depth = event["depth_km"]
    event_line = f"{event_time}{abs(lat_degree):2d}{south}{abs(lat_minute):4.0f}{abs(lng_degree):3d}{east}{abs(lng_minute):4.0f}{depth:5.0f}"
    out_file.write(event_line + "\n")

    picks_idx = picks_by_event[event["match_id"]]
    for j in picks_idx:
        pick = picks.iloc[j]
        network_code, station_code, comp_code, channel_code = pick['id'].split('.')
        phase_type = pick['type']
        phase_weight = min(max(int((1 - pick['prob']) / (1 - 0.3) * 4) - 1, 0), 3)
        pick_time = datetime.strptime(pick["timestamp"], "%Y-%m-%dT%H:%M:%S.%f")
        phase_time_minute = pick_time.strftime("%Y%m%d%H%M")
        phase_time_second = pick_time.strftime("%S%f")[:-4]
        tmp_line = f"{station_code:<5}{network_code:<2} {comp_code:<1}{channel_code:<3}"
        if phase_type.upper() == 'P':
            pick_line = f"{tmp_line:<13} P {phase_weight:<1d}{phase_time_minute} {phase_time_second}"
        elif phase_type.upper() == 'S':
            pick_line = f"{tmp_line:<13}   4{phase_time_minute} {'':<12}{phase_time_second} S {phase_weight:<1d}"
        else:
            raise (f"Phase type error {phase_type}")
        out_file.write(pick_line + "\n")

    out_file.write("\n")
    if i > 22e3:
        break

out_file.close()

100%|█████████████████████████████████████████████████████████████████████████████| 1897/1897 [00:14<00:00, 128.23it/s]


In [4]:
pick

id                            PB.B921..EH
timestamp      2019-07-04T19:59:10.074000
prob                              0.79263
amp                              0.000022
type                                    S
event_index                          1767
gamma_score                      0.121487
match_id                        1767_1767
Name: 66482, dtype: object

# Hypoinverse to HypoDD

In [5]:
def convert_stations():
    os.system("python convert_stations.py")
def convert_picks():

    picks = pd.read_csv(data_path("gamma_picks.csv"))
    events = pd.read_csv(data_path("gamma_events.csv"))

    if "file_index" in events.columns:
        events["match_id"] = events.apply(lambda x: f'{x["event_index"]}_{x["file_index"]}', axis=1)
        picks["match_id"] = picks.apply(lambda x: f'{x["event_index"]}_{x["file_index"]}', axis=1)
    elif "file_idx" in events.columns:
        events["match_id"] = events.apply(lambda x: f'{x["event_index"]}_{x["file_idx"]}', axis=1)
        picks["match_id"] = picks.apply(lambda x: f'{x["event_index"]}_{x["file_idx"]}', axis=1)
    else:
        events["match_id"] = events["event_index"]
        picks["match_id"] = picks["event_index"]
    events.sort_values(by="time", inplace=True, ignore_index=True)

    # %%
    # MAXEVENT = len(events)
    MAXEVENT = 2e5  ## segment by time
    MAXEVENT = len(events) // ((len(events) - 1) // MAXEVENT + 1) + 1

    # %% convert format
    idx = 0
    out_file = open(f"results/hypoDD.pha", "w")
    hypoDD_catalogs = []

    picks_by_event = picks.groupby("match_id").groups
    for i in tqdm(range(len(events))):
        if i % MAXEVENT == MAXEVENT - 1:
            out_file.close()
            idx = int(i // MAXEVENT)
            os.system(f"mkdir tmp_{idx:02d}")
            os.system(f"cp hypoDD.pha hypoDD.inp ph2dt.inp stations_hypoDD.dat ./tmp_{idx:02d}")
            # catalog = run_hypoDD(i, MAXEVENT)
            # hypoDD_catalogs.append(catalog)
            out_file = open("results/hypoDD.pha", "w")

        event = events.iloc[i]
        event_time = datetime.strptime(event["time"], "%Y-%m-%dT%H:%M:%S.%f")
        lat = event["latitude"]
        lng = event["longitude"]
        dep = event["depth_km"]
        mag = event["magnitude"]
        EH = 0
        EZ = 0
        #RMS = float(event["covariance"].split(",")[0])
        RMS = float(str(event["cov_time_amp"]).split(",")[0])
        if RMS > 10:
            RMS = 0
        year, month, day, hour, min, sec = (
            event_time.year,
            event_time.month,
            event_time.day,
            event_time.hour,
            event_time.minute,
            float(event_time.strftime("%S.%f")),
        )
        event_line = f"# {year:4d} {month:2d} {day:2d} {hour:2d} {min:2d} {sec:5.2f}  {lat:7.4f} {lng:9.4f}   {dep:5.2f} {mag:5.2f} {EH:5.2f} {EZ:5.2f} {RMS:5.2f} {i+1:9d}\n"
        out_file.write(event_line)

        picks_idx = picks_by_event[event["match_id"]]
        for j in picks_idx:
            pick = picks.iloc[j]
            network_code, station_code, comp_code, channel_code = pick['id'].split('.')
            phase_type = pick['type'].upper()
            phase_weight = pick['prob']
            pick_time = (datetime.strptime(pick["timestamp"], "%Y-%m-%dT%H:%M:%S.%f") - event_time).total_seconds()
            # if pick_time <= 0:
            #     continue
            pick_line = f"{station_code:<5s}    {pick_time:8.3f}   {phase_weight:5.4f}   {phase_type}\n"
            out_file.write(pick_line)

    out_file.close()
    idx = int(i // MAXEVENT)
    os.system(f"mkdir tmp_{idx:02d}")
    os.system(f"cp hypoDD.pha hypoDD.inp ph2dt.inp stations_hypoDD.dat ./tmp_{idx:02d}")

In [6]:
print(f"Start time: {time.asctime()}")
start_time = time.time()
convert_stations()
convert_picks()

Start time: Sat Jan 20 19:31:12 2024


100%|█████████████████████████████████████████████████████████████████████████████| 1897/1897 [00:12<00:00, 152.08it/s]
